In [17]:
import pyodbc
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
from keras.models import load_model
import random
import numpy as np
import tensorflow as tf

server = 'cyse492group8.database.windows.net'
database = 'CYSE492_Project'
username = 'cyseadmin'
password = 'GroupProject492'
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

In [4]:
cursor.execute("SELECT [Control Identifier],[Control Text]  FROM [dbo].['NIST_800.53_controls$']")

labels = []
texts = []
for row in cursor:
    texts.append(row[1])
    if 'AC' in row[0]:
        labels.append(1)
    if 'AT' in row[0]:
        labels.append(2)
    if 'AU' in row[0]:
        labels.append(3)
    if 'CA' in row[0]:
        labels.append(4)
    if 'CM' in row[0]:
        labels.append(5)
    if 'CP' in row[0]:
        labels.append(6)
    if 'IA' in row[0]:
        labels.append(7)
    if 'IR' in row[0]:
        labels.append(8)
    if 'MA' in row[0]:
        labels.append(9)
    if 'MP' in row[0]:
        labels.append(10)
    if 'PE' in row[0]:
        labels.append(11)
    if 'PL' in row[0]:
        labels.append(12)
    if 'PM' in row[0]:
        labels.append(13)
    if 'PS' in row[0]:
        labels.append(14)
    if 'PT' in row[0]:
        labels.append(15)
    if 'RA' in row[0]:
        labels.append(16)
    if 'SA' in row[0]:
        labels.append(17)
    if 'SC' in row[0]:
        labels.append(18)
    if 'SI' in row[0]:
        labels.append(19)
    if 'SR' in row[0]:
        labels.append(20)

x_test = []
y_test = []
x_act = np.array(texts)
y_act = np.array(labels)
print(len(labels))

1189


In [5]:
print(len((x_act)))
print(len((y_act)))

1189
1189


In [6]:
#Manually mapping cis

cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES")

#Stuff Im giving to the model
cisNames = ["'CIS_Microsoft_Intune_for_Windows_10_Benchmark_v2.0.0 Level 1 (L1) - Corporate_Enter$'","'CIS_Apple_macOS_11.0_Benchmark_v1.2.0 Level 1$'",
            "'CIS_Apple_iOS_16_and_iPadOS_16_Benchmark_v1.1.0-EXCEL$'","'Prototype CIS Microsoft Intune for Windows 10 Release 2004 Benchmark v1.0.1_ARCHIVE Level 1 (L1)$'",
           "'Prototype_CIS_Microsoft_Windows_Server_2019_STIG_Benchmark_v1.0.0 Combined Profiles$'","'CIS_Ubuntu_Linux_20.04_LTS_STIG_Benchmark_v2.0.0 Level 1 - Server$'",
           "'CIS_SOPHOS_XG_Firewall_v18_Benchmark_v1.0.0_Level1$'","'CIS_Palo_Alto_Firewall_10_Benchmark_v1.0.0-FIXED_Level1$'","'CIS_Microsoft_Internet_Explorer_10_Benchmark_v1.0.0_ARCHIVE$'",
           "'CIS_Apple_macOS_10.15_Catalina_Benchmark_v3.0.0 Combined Profiles$'","'CIS_Check_Point_Firewall_Benchmark_v1.1.0_Level1$'","'CIS_Cisco_Firewall_Benchmark_v4.1.0_Certification_Level1$'",
           "'CIS_Fortigate_7.0.x_Benchmark_v1.2.0$'","'CIS_pfSense_Firewall_Benchmark_v1.1.0$'","'CIS_Microsoft_Windows_Server_2022_Benchmark_v2.0.0 Combined Profiles$'",
           "'CIS_Apple_iOS_16_and_iPadOS_16_Benchmark_v1.1.0-EXCEL$'","'CIS_Cisco_NX-OS_Benchmark_v1.0.0 EXCEL_Level2$'",
            "'CIS_Cisco_IOS_17.x_Benchmark_v2.0.0$'","'CIS_Apple_iOS_15_and_iPadOS_15_Benchmark_v1.1.0$'"]

#labels is the section num, texts is the description

#Get individual tables
setCount=0
for count in range(len(cisNames)):
    try:
        cursor.execute("SELECT [Section #], [Description]  FROM [dbo].["+cisNames[count]+"]")
        #checks and sorts by first number in the section number
        for row in cursor:
            if row[0] != None and row[1] != None and "This section is intentionally blank" not in row[1]:
                texts.append(str(row[1]))            
                if str(row[0])[0:2][-1] == ".":
                    labels.append(str(int(float(str(row[0])[0:3]))))#setCount)
                elif str(row[0])[0:3][-1] == ".":
                    labels.append(str(int(float(str(row[0])[0:4]))))#setCount)
                else:
                    labels.append(str(int(float(str(row[0])[0:2]))))#setCount)
            if len(texts) != len(labels):
                print("WARNNNING THEY FELL OFF HERE!:"+str(cisNames[count])+"      "+str(row[0])+" "+str(row[1]))
    except pyodbc.ProgrammingError:
        print(cisNames[count]+" is missing either description or label. Skipping")
x_act = np.array(texts)
y_act = np.array(labels)
print(len(labels))

5153


In [143]:
# Hyperparameters
max_words = 50000 # max number of words to use in the vocabulary was 15k
max_len = 400 # max length of each text (in terms of number of words) 250
embedding_dim = 400 # dimension of word embeddings was 100
lstm_units = 32 # number of units in the LSTM layer
# num_classes = len(set(labels)) + 1 # number of classes
num_classes = len(set(y_act)) + 1 # number of classes
#num_classes2 = len(set(y_test)) + 1

# Tokenize the texts and create a vocabulary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_act)
sequences = tokenizer.texts_to_sequences(x_act)

# Pad the sequences so they all have the same length
x = pad_sequences(sequences, maxlen=max_len)


# Create one-hot encoded labels
y = keras.utils.to_categorical(y_act, num_classes)

# y_test = keras.utils.to_categorical(y_test, num_classes)
# Build the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(LSTM(lstm_units))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(x, y, batch_size=32, epochs=20)# was 32,20


Epoch 1/20
162/162 [==============================] - 108s 632ms/step - loss: 2.1271 - accuracy: 0.3759
Epoch 2/20
162/162 [==============================] - 75s 464ms/step - loss: 1.2298 - accuracy: 0.6641
Epoch 3/20
162/162 [==============================] - 54s 335ms/step - loss: 0.7710 - accuracy: 0.7858
Epoch 4/20
162/162 [==============================] - 57s 351ms/step - loss: 0.5423 - accuracy: 0.8465
Epoch 5/20
162/162 [==============================] - 60s 373ms/step - loss: 0.3908 - accuracy: 0.8851
Epoch 6/20
162/162 [==============================] - 62s 385ms/step - loss: 0.3130 - accuracy: 0.9045
Epoch 7/20
162/162 [==============================] - 60s 371ms/step - loss: 0.2542 - accuracy: 0.9253
Epoch 8/20
162/162 [==============================] - 59s 365ms/step - loss: 0.1964 - accuracy: 0.9365
Epoch 9/20
162/162 [==============================] - 60s 373ms/step - loss: 0.1609 - accuracy: 0.9462
Epoch 10/20
162/162 [==============================] - 60s 368ms/step - 

In [144]:
#we like training
# Tokenize the texts and create a vocabulary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_act)
sequences = tokenizer.texts_to_sequences(x_act)

# Pad the sequences so they all have the same length
x = pad_sequences(sequences, maxlen=max_len)


# Create one-hot encoded labels
y = keras.utils.to_categorical(y_act, num_classes)

for loop10 in range(2):
    history = model.fit(x, y, batch_size=32, epochs=20)# was 32,20

Epoch 1/20
162/162 [==============================] - 61s 374ms/step - loss: 0.0600 - accuracy: 0.9676
Epoch 2/20
162/162 [==============================] - 61s 377ms/step - loss: 0.0678 - accuracy: 0.9637
Epoch 3/20
162/162 [==============================] - 60s 368ms/step - loss: 0.1421 - accuracy: 0.9474
Epoch 4/20
162/162 [==============================] - 58s 358ms/step - loss: 0.0867 - accuracy: 0.9610
Epoch 5/20
162/162 [==============================] - 59s 361ms/step - loss: 0.0698 - accuracy: 0.9645
Epoch 6/20
162/162 [==============================] - 58s 359ms/step - loss: 0.0532 - accuracy: 0.9676
Epoch 7/20
162/162 [==============================] - 59s 366ms/step - loss: 0.0512 - accuracy: 0.9690
Epoch 8/20
162/162 [==============================] - 59s 365ms/step - loss: 0.0585 - accuracy: 0.9680
Epoch 9/20
162/162 [==============================] - 60s 373ms/step - loss: 0.0501 - accuracy: 0.9670
Epoch 10/20
162/162 [==============================] - 58s 360ms/step - l

In [146]:
model.save('CYSE493_Group8_CISNISTEdit_Model.keras')

In [7]:
model = load_model('CYSE493_Group8_CISNISTEdit_Model.keras')

In [8]:
oldCis = cisNames

In [9]:
print(len(oldCis))

19


In [74]:
###padding
## reuse old tokenizer for x_test
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_act)
sequences = tokenizer.texts_to_sequences(x_test)

## Pad the sequences so they all have the same length
x_test = pad_sequences(sequences, maxlen=max_len)
y_test = keras.utils.to_categorical(y_test, num_classes)

#Test the model     
#If you want to test different x_test/y_test you can compile that section instead of retraining the whole thing again
accr = model.evaluate(x_test,y_test) #x and y instead of test?
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

#for i in range(10):
#    result = tf.argmax(model.predict(tf.expand_dims(texts[i], 0)), axis=1)
#    print(result.numpy(), labels[i])

ValueError: Expected input data to be non-empty.

In [10]:
#Train with CIS
#Get names of tables
cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES")

cisNames = []
for names in cursor:
    if names[0]!=("'NIST_800.53_controls$'") and names[0]!="'NIST_800.53_baselines$'" and names[0] not in oldCis:
        cisNames.append(names[0])

#labels is the section num, texts is the description
labels = []
texts = []
#Get individual tables
setCount=0
for count in range(len(cisNames)):
    try:
        cursor.execute("SELECT [Section #], [Description]  FROM [dbo].["+cisNames[count]+"]")
        #checks and sorts by first number in the section number
        for row in cursor:
            if row[0] != None and row[1] != None and "This section is intentionally blank" not in row[1]:
                texts.append(str(row[1]))            
                if str(row[0])[0:2][-1] == ".":
                    labels.append(str(int(float(str(row[0])[0:3]))))#setCount)
                elif str(row[0])[0:3][-1] == ".":
                    labels.append(str(int(float(str(row[0])[0:4]))))#setCount)
                else:
                    labels.append(str(int(float(str(row[0])[0:2]))))#setCount)
            if len(texts) != len(labels):
                print("WARNNNING THEY FELL OFF HERE!:"+str(cisNames[count])+"      "+str(row[0])+" "+str(row[1]))
    except pyodbc.ProgrammingError:
        print(cisNames[count]+" is missing either description or label. Skipping")

x_test = []
y_test = []
x_cis = np.array(texts)
y_cis = np.array(labels)
y_cis = y_cis.astype(float)
y_cis = y_cis.astype(int)

# test is the actual test, act is used for the training
# for i in range(150):
#     random_index = random.randrange(len(labels))
#     x_test.append(texts.pop(random_index))
#     y_test.append(labels.pop(random_index))
# x_test = np.array(x_test)
# y_test = np.array(y_test)
# y_test = y_test.astype(float)
# y_test = y_test.astype(int)

'MITRE ATT&CK Filtering$' is missing either description or label. Skipping
'MITRE ATT&CK Filtering$'_xlnm#_FilterDatabase is missing either description or label. Skipping
CIS_Apple_macOS_12.0_Monterey_Benchmark_v3.0.0 License$ is missing either description or label. Skipping
'MITRE ATT&CK Mappings$' is missing either description or label. Skipping
'CIS_Apple_macOS_12.0_Monterey_Benchmark_v3.0.0 MITRE ATT&CK Filtering$' is missing either description or label. Skipping
'CIS_Apple_macOS_12.0_Monterey_Benchmark_v3.0.0 MITRE ATT&CK Filtering$'_xlnm#_FilterDatabase is missing either description or label. Skipping
'CIS_Apple_macOS_12.0_Monterey_Benchmark_v3.0.0 MITRE ATT&CK Mappings$' is missing either description or label. Skipping
'CIS_Apple_macOS_12.0_Monterey_Benchmark_v3.0.0 Overview - Glossary$' is missing either description or label. Skipping
CIS_Microsoft_Windows_Server_2019_STIG_Benchmark_v1.1.0 License$ is missing either description or label. Skipping
CIS_Apple_macOS_11.0_Big_Sur_Be

In [11]:
print(len((x_act)))
print(len((y_act)))
print(len((x_cis)))
print(len((y_cis)))
print(len(cisNames))
print(cisNames[13])
# for dist in y_act:
     # print(dist)

5153
5153
41055
41055
278
CIS_Microsoft_Windows_Server_2019_STIG_Benchmark_v1.1.0 Combined$


In [12]:
#model.save('CYSE493_Group8_CIS-NIST_Model.keras')  # creates a keras save file
# identical to the previous one
#model = load_model('CYSE493_Group8_CIS-NIST_Model.keras')
print(len(set(y_act)))

20


In [14]:
#new tests
###padding
num_classes = len(set(y_act)) + 1 # number of classes

## reuse old tokenizer for x_test
tokenizer = Tokenizer(num_words=50000)#max_words)
tokenizer.fit_on_texts(x_act)#x_cis
sequences = tokenizer.texts_to_sequences(x_cis)

## Pad the sequences so they all have the same length
x_test = pad_sequences(sequences, maxlen=400)
y_test = keras.utils.to_categorical(y_cis, num_classes)

#Test the model     
#If you want to test different x_test/y_test you can compile that section instead of retraining the whole thing again
accr = model.evaluate(x_test,y_test) #was x_test and y_test?
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

#for i in range(10):
#    result = tf.argmax(model.predict(tf.expand_dims(texts[i], 0)), axis=1)
#    print(result.numpy(), labels[i])



1283/1283 [==============================] - 80s 61ms/step - loss: 0.5008 - accuracy: 0.9204
Test set
  Loss: 0.501
  Accuracy: 0.920


In [ ]:
print(len(set(y_act)))